# Advent of Code 2021

Imports

In [1]:
import math
import os
from typing import List, Union

import numpy as np
import pandas as pd

General purpose functions

In [2]:
# Read in puzzle inputs from `data` folder at root of repo.
# Files containing puzzle inputs are formatted as 2021_day_<day number>_input.txt
INPUT_DIR = 'data'
INPUT_FN_TEMPLATE = '2021_day_{}_input.txt'

def get_puzzle_input(day):
    """load puzzle input for a given day from the `data` folder at the root of the repo
    """
    fp = os.path.join(INPUT_DIR, INPUT_FN_TEMPLATE.format(day))
    with open(fp, 'r') as f:
        return [ln.strip() for ln in f.readlines()]

---
**Day 1: Part 1**

link to puzzle [here](https://adventofcode.com/2021/day/1)

Functions

In [3]:
def count_increasing_depths(data: List[Union[int, float]]) -> float:
    """given 
    """
    arr = np.array(data)
    return (arr[:-1] < arr[1:]).sum()

Solution

In [4]:
data = get_puzzle_input(day=1)
print(count_increasing_depths(data))

1194


#### Part 2

Functions

In [5]:
def window_sum(data, window):
    df = pd.DataFrame(data, columns=['depth'])
    return df.depth.rolling(window).sum().tolist()

Solution

In [6]:
print(count_increasing_depths(window_sum(data, 3)))

1235


---
## Day 2
#### Part 1

Functions

In [7]:
def directional_command_to_xy(s):
    """turn a direction command string into its representation as a vector translation in the 
    x,y plane of the form [x, y] such that adding the coordinate to another x,y pair [p1, p2] 
    representing current position, then [p1 + x, p2 + y] is the position resulting from performing 
    the directional command. 
    
    E.g., if our current position is [0, 0] and we receive the directional command `forward 8`, 
    then we would return the coordinate vector [8, 0], since moving our current position [0, 0]
    8 units along the x axis and 0 units along the y axis restults in [8, 0] = [0 + 8, 0 + 0].
    """
    direction, units = tuple(s.split())
    # increase horizontal position
    if direction == 'forward':  
        xy = np.array([int(units), 0])
    # increase depth
    elif direction == 'down':  
        xy = np.array([0, int(units)])
    # decrease depth
    elif direction == 'up':
        xy = np.array([0, -int(units)])
    # if direction is none of forward, down, up, then return None and print a warning
    # that the direction command is unknown
    else:
        print(f'Unknown direction command: {direction}')
        xy = None
    return xy

def final_position(data):
    """compute the final position resulting from a series of directional commands by summing
    their representations as vector translations in the x,y plane.
    """
    return np.array([directional_command_to_xy(s) for s in data]).sum(axis=0)

Solution

In [8]:
data = get_puzzle_input(day=2)
print(math.prod(final_position(data)))

1459206


#### Part 2

Functions

In [9]:
def compute_final_position_aim(commands):
    """
    """
    pos = [0, 0, 0]
    for cmd in commands:
        direction = cmd.split()[0]
        units = int(cmd.split()[1])
        
        # increase horizontal position
        if direction == 'forward':
            pos[0] = pos[0] + units
            pos[1] = pos[1] + pos[2] * units
            
        # increase depth
        elif direction == 'down':  
            pos[2] = pos[2] + units
        
        # decrease depth
        elif direction == 'up':
            pos[2] = pos[2] - units

        # if direction is none of forward, down, up, then return None and print a warning
        # that the direction command is unknown
        else:
            raise(ValueError(f'Unknown direction command: {direction}'))
    return pos

Solution

In [10]:
print(math.prod(compute_final_position_aim(data)[:-1]))

1320534480


---
## Day 3
#### Part 1

Functions

In [99]:
def binary_array_to_string(arr):
    """turn an 1 x n array `arr` whose elements are binary digits, i.e. 1 or 0
    into a string `s` such that `s[i] = str(arr[i])`
    """
    return ''.join(arr.astype(str))

def binary_strings_to_array(data):
    """turn a list of binary strings into a numpy array as follows. If there are m binary strings of length n, 
    then create an m x n table where the element at row i, column j is the jth digit of the ith 
    binary string, as an int
    """
    return np.array([list(b) for b in data]).astype(int)

def get_least_frequent_bits(arr):
    """for an m x n array `arr` in {0,1}^(m x n), return a 1 x n array in {1,0}^n
    whose ith position is the bit which appears **least** frequently in ith column of `arr`,
    with ties going to 0
    """
    # get frequency of 1s and 0s in each column
    freq_ones = arr.sum(axis=0)
    freq_zeros = arr.shape[0] - freq_ones
    
    # find least frequent bit for each column by doing element-wise comparison
    least_frequent_bits = (freq_ones < freq_zeros).astype(int)

    # when frequencies are equal, set to 0
    least_frequent_bits[freq_ones == freq_zeros] = 0
    return least_frequent_bits

def get_most_frequent_bits(arr):
    """for an m x n array `arr` in {0,1}^(m x n), return a 1 x n array in {1,0}^n
    whose ith position is the bit which appears **most** frequently in ith column of `arr`,
    with ties going to 1
    """
    # get frequency of 1s and 0s in each column
    freq_ones = arr.sum(axis=0)
    freq_zeros = arr.shape[0] - freq_ones
    
    # find least frequent bit for each column by doing element-wise comparison
    most_frequent_bits = (freq_ones > freq_zeros).astype(int)

    # when frequencies are equal, set to 0
    most_frequent_bits[freq_ones == freq_zeros] = 1
    return most_frequent_bits

def get_gamma_rate(arr):
    return int(binary_array_to_string(get_most_frequent_bits(arr)), base=2)

def get_epsilon_rate(arr):
    return int(binary_array_to_string(get_least_frequent_bits(arr)), base=2)

Solution

In [116]:
data = get_puzzle_input(day=3)
arr = binary_strings_to_array(data)
gamma = get_gamma_rate(arr)
epsilon = get_epsilon_rate(arr)
print(gamma * epsilon)

1082324


#### Part 2

Functions

In [121]:
def get_rating(arr, bit_criteria):
    ncols = arr.shape[1]
    rating_arr = arr[:, :]
    for i in range(ncols):
        rating_arr = rating_arr[bit_criteria(rating_arr, i), :]
        if rating_arr.shape[0] == 1:
            break
    return int(binary_array_to_string(rating_arr[0]), base=2)

def oxygen_generator_rating_bit_criteria(arr, i):
    return arr[:, i] == get_least_frequent_bits(arr)[i]

def CO2_scrubber_rating_bit_criteria(arr, i):
    return arr[:, i] == get_most_frequent_bits(arr)[i]

def get_CO2_scrubber_rating(arr):
    return get_rating(arr, CO2_scrubber_rating_bit_criteria)

def get_oxygen_generator_rating(arr):
    return get_rating(arr, oxygen_generator_rating_bit_criteria)

def get_life_support_rating(arr):
    return get_oxygen_generator_rating(arr) * get_CO2_scrubber_rating(arr)

Solution

In [122]:
print(get_life_support_rating(arr))

1353024
